In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
import math
import numpy as np

Get sorted cases id

In [3]:
import wget
import os.path

In [18]:
df_final = pd.read_csv('BPI_12_anonimyzed.csv')

In [19]:
df_final.columns

Index(['Case ID', 'Activity', 'Complete Timestamp', 'Resource',
       '(case) AMOUNT_REQ', 'concept:name'],
      dtype='object')

In [20]:
df_final.head()

,Case ID,Activity,Complete Timestamp,Resource,(case) AMOUNT_REQ,concept:name
0,1,a1,2011-10-01 11:45:13,NaN,20000,W_Completeren aanvraag
1,1,a2,2011-10-01 12:17:08,NaN,20000,W_Nabellen offertes
2,1,a2,2011-10-08 16:32:00,10913.0,20000,W_Nabellen offertes
3,1,a2,2011-10-10 11:33:05,11049.0,20000,W_Nabellen offertes
4,1,a3,2011-10-13 10:37:37,10629.0,20000,W_Valideren aanvraag


In [21]:
df_final['Complete Timestamp']

0        2011-10-01 11:45:13
1        2011-10-01 12:17:08
2        2011-10-08 16:32:00
3        2011-10-10 11:33:05
4        2011-10-13 10:37:37
                ...         
72408    2012-03-01 09:28:21
72409    2012-03-01 09:27:52
72410    2012-03-01 20:22:41
72411    2012-03-10 12:50:54
72412    2012-03-01 09:27:41
Name: Complete Timestamp, Length: 72413, dtype: object

In [22]:
sorted_time_cases = df_final.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values

In [23]:
df_final.shape

(72413, 6)

In [24]:
df_final.columns

Index(['Case ID', 'Activity', 'Complete Timestamp', 'Resource',
       '(case) AMOUNT_REQ', 'concept:name'],
      dtype='object')

In [25]:
df_final['Activity'].unique()

array(['a1', 'a2', 'a3', 'a4', 'a5', 'a6'], dtype=object)

Example:

In [26]:
df_final

,Case ID,Activity,Complete Timestamp,Resource,(case) AMOUNT_REQ,concept:name
0,1,a1,2011-10-01 11:45:13,NaN,20000,W_Completeren aanvraag
1,1,a2,2011-10-01 12:17:08,NaN,20000,W_Nabellen offertes
2,1,a2,2011-10-08 16:32:00,10913.0,20000,W_Nabellen offertes
3,1,a2,2011-10-10 11:33:05,11049.0,20000,W_Nabellen offertes
4,1,a3,2011-10-13 10:37:37,10629.0,20000,W_Valideren aanvraag
...,...,...,...,...,...,...
72408,9656,a4,2012-03-01 09:28:21,11169.0,20000,W_Afhandelen leads
72409,9657,a4,2012-03-01 09:27:52,11169.0,8500,W_Afhandelen leads
72410,9657,a1,2012-03-01 20:22:41,10933.0,8500,W_Completeren aanvraag
72411,9657,a2,2012-03-10 12:50:54,11119.0,8500,W_Nabellen offertes


In [27]:
number_cases = len(df_final['Case ID'].unique())

In [28]:
number_cases

9658

Spliting data (train, test, validation)

In [29]:
from sklearn.model_selection import train_test_split

train_cases, test_cases  = train_test_split(sorted_time_cases, test_size=0.4, 
                                            shuffle=False, 
                                            random_state=42
                                           )

test_cases, val_cases  = train_test_split(test_cases, test_size=0.5, 
                                            shuffle=False, 
                                            random_state=42
                                           )

In [30]:
len(train_cases), len(test_cases), len(val_cases)

(5794, 1932, 1932)

In [31]:
len(train_cases) / number_cases

0.5999171671153448

Dataframes creation

In [32]:
df_train = df_final[df_final['Case ID'].isin(train_cases)].reset_index().drop(['index'], axis=1)
df_test = df_final[df_final['Case ID'].isin(test_cases)].reset_index().drop(['index'], axis=1)
df_val = df_final[df_final['Case ID'].isin(val_cases)].reset_index().drop(['index'], axis=1)

In [33]:
df_train.shape

(43999, 6)

Overriding cases checking

In [34]:
print(
    set(test_cases).intersection(set(train_cases)),
    set(test_cases).intersection(set(val_cases)),
    set(train_cases).intersection(set(val_cases))
)

set() set() set()


In [35]:
df_sorted = pd.concat([df_train, df_val, df_test])

In [36]:
df_sorted.head()

,Case ID,Activity,Complete Timestamp,Resource,(case) AMOUNT_REQ,concept:name
0,1,a1,2011-10-01 11:45:13,NaN,20000,W_Completeren aanvraag
1,1,a2,2011-10-01 12:17:08,NaN,20000,W_Nabellen offertes
2,1,a2,2011-10-08 16:32:00,10913.0,20000,W_Nabellen offertes
3,1,a2,2011-10-10 11:33:05,11049.0,20000,W_Nabellen offertes
4,1,a3,2011-10-13 10:37:37,10629.0,20000,W_Valideren aanvraag


In [37]:
df_sorted.to_csv('BPI_12_anonimyzed_sorted.csv', index=False)